In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import glob
import re
from tkinter import filedialog as fd
from tkinter import *
from scipy.stats import lognorm

%matplotlib inline

In [ ]:
def Gaussian(x, a, Mean, Sigma):
    return a * np.exp(-((x - Mean)**2 / (2 * Sigma**2)))

def LogNorm(x, a, Mu, Sigma):
    return a/(x*Sigma*np.sqrt(2*np.pi))*np.exp(-(np.log(x)-Mu)**2/(2.0*Sigma**2))

In [ ]:
def readParticleConfig(path):
    with open(path,'r') as F:
        Data = F.read()
    Data = Data.strip()
    FirstRow = Data.split(sep='\n')[0]
    NumberOfAParticles = int(FirstRow.split()[6].strip('|'))
    NumberOfBParticles = int(FirstRow.split()[8].strip('|'))
    ListOfRows = Data.split(sep='\n')[1:]
    if (len(ListOfRows) > 0):
        NumberOfColumns = len(ListOfRows[0].split()) - 2 
    else:
        NumberOfColumns = 0
    APositions = np.empty((NumberOfAParticles, NumberOfColumns))
    BPositions = np.empty((NumberOfBParticles, NumberOfColumns))
    AIndex = 0
    BIndex = 0
    
    for i in range(len(ListOfRows)):
        if (ListOfRows[i].split()[NumberOfColumns + 1] == 'A'):
            for j in range(NumberOfColumns):
                APositions[AIndex,j] = ListOfRows[i].split()[j+1]
            AIndex += 1
        else:
            for j in range(NumberOfColumns):
                BPositions[BIndex,j] = ListOfRows[i].split()[j+1]
            BIndex += 1
    return APositions, BPositions

def getFilePathFromDialog():
    root = Tk()
    FilePath = fd.askopenfilename()
    root.destroy()
    return FilePath

def getDirectoryPathFromDialog():
    root = Tk()
    Directory = fd.askdirectory()
    root.destroy()
    return Directory

def readParticleConfigFromDialog():
    return readParticleConfig(getFilePathFromDialog())

In [ ]:
def readSeries(path):
    with open(path,'r') as F:
        Data = F.read()
    Data = Data.strip()
    ListOfData = Data.split(sep='\n')[1:]
    NumberOfEntries = len(ListOfData)
    Series = np.empty(NumberOfEntries)
    for i in range(NumberOfEntries):
        Series[i] = ListOfData[i]
    return Series

def readSeriesFromDialog():
    return readSeries(getFilePathFromDialog())

In [ ]:
def computeEquilibriumAverage(Series):
    n = 0
    while (n < len(Series)):
        Average = np.mean(Series[n:])
        Deviations = np.abs(Average-Series[n:])
        AverageDeviation = np.mean(Deviations)
        if (np.abs(Average-Series[n]) < AverageDeviation):
            return Average,AverageDeviation,n
        n += 1
    return np.mean(Series[:]), np.mean(np.abs(np.mean(Series[:])-Series[:])), 0

## Plot particle positions

In [ ]:
APositions, BPositions = ReadParticleConfigFromDialog()

In [ ]:
Fig, Host = plt.subplots(1,1,squeeze=False,figsize=(5,5),dpi =100)
Host[0,0].plot(APositions[:,0],APositions[:,1],'.',color='b')
Host[0,0].plot(BPositions[:,0],BPositions[:,1],'.',color='darkorange')
Host[0,0].set_xlabel('x')
Host[0,0].set_ylabel('y')
Host[0,0].set_xlim(0,1)
Host[0,0].set_ylim(0,1)

## Series of particle number

In [ ]:
NumberOfASeries = ReadSeriesFromDialog()

In [ ]:
NumberOfAAverage, AverageNumberDeviation, NumberIndex = computeEquilibriumAverage(NumberOfASeries)
print(NumberOfAAverage, AverageNumberDeviation, NumberIndex)

In [ ]:
Fig, Host = plt.subplots(1,1,squeeze=False, figsize = (12,5), dpi = 100)
Host[0,0].plot(NumberOfASeries,'.')
Host[0,0].plot(np.ones(len(NumberOfASeries))*NumberOfAAverage,color='darkorange',label='Gleichgewichts-Mittel')
Host[0,0].plot(np.ones(len(NumberOfASeries))*NumberOfAAverage+AverageNumberDeviation,color='darkorange',alpha= 0.5)
Host[0,0].plot(np.ones(len(NumberOfASeries))*NumberOfAAverage-AverageNumberDeviation,color='darkorange',alpha = 0.5)
Host[0,0].vlines(NumberIndex, min(NumberOfASeries), max(NumberOfASeries), color = 'k', alpha = 0.5)

## Series of potential energy

In [ ]:
PotEnergySeries = ReadSeriesFromDialog()

In [ ]:
PotEnergyAverage, AverageEnergyDeviation, EnergyIndex = computeEquilibriumAverage(PotEnergySeries)
print(PotEnergyAverage, AverageEnergyDeviation, EnergyIndex)

In [ ]:
Fig, Host = plt.subplots(1,1,squeeze=False, figsize = (12,5), dpi = 100)
Host[0,0].plot(PotEnergySeries,'.')
Host[0,0].plot(np.ones(len(PotEnergySeries))*PotEnergyAverage,color='darkorange',label='Gleichgewichts-Mittel')
Host[0,0].plot(np.ones(len(PotEnergySeries))*PotEnergyAverage+AverageEnergyDeviation,color='darkorange',alpha= 0.5)
Host[0,0].plot(np.ones(len(PotEnergySeries))*PotEnergyAverage-AverageEnergyDeviation,color='darkorange',alpha = 0.5)
Host[0,0].vlines(EnergyIndex, min(PotEnergySeries), max(PotEnergySeries), color = 'k', alpha = 0.5)

## Gaussian fits

In [ ]:
def computeHist(Series):
    Min = int(np.min(Series))
    Max = int(np.max(Series))
    yValues = np.zeros(Max - Min + 1)
    xValues = np.arange(Min,Max + 1)
    for Entry in Series:
        yValues[int(Entry)-Min] += 1
    return yValues,xValues

def smoothCurve(RawCurve,WindowLength):
    SmoothedArray = np.zeros(len(RawCurve))
    OffsetMax = WindowLength//2
    for i in range(len(RawCurve)):
        Average = 0.0
        NumberOfDataPoints = 0
        for Offset in np.arange(-OffsetMax, OffsetMax, 1):
            if (i+Offset >= 0 and i+Offset < len(RawCurve)):
                Average += RawCurve[i+Offset]
                NumberOfDataPoints += 1
        SmoothedArray[i] = Average/NumberOfDataPoints
    return SmoothedArray

def lumpData(yData,xData,WindowLength):
    LumpedyData = np.zeros(len(yData) // WindowLength)
    LumpedxData = np.zeros(len(yData) // WindowLength)
    i = 0
    Index = 0
    for Index in range(len(LumpedyData)):
        LumpedxData[Index] = xData[i]+WindowLength/2
        NewAverage = 0.0
        NumberOfDataPoints = 0
        for j in range(WindowLength):
            if (i+j < len(yData)):
                NewAverage += yData[i+j]
                NumberOfDataPoints += 1
        LumpedyData[Index] = NewAverage/NumberOfDataPoints
        i += WindowLength
    return LumpedyData, LumpedxData

def computeLocalMaxima(LumpedyData,LumpedxData):
    LocalMaxima = []
    for i in range(len(LumpedData)):
        if (i-1 >= 0 and i+1 < len(LumpedyData)):
            if (LumpedyData[i-1] < LumpedyData[i] and LumpedyData[i+1] < LumpedyData[i]):
                LocalMaxima.append(LumpedxData[i])
    return LocalMaxima

In [ ]:
yValues, xValues = computeHist(NumberOfASeries[NumberIndex:])

In [ ]:
GaussParameters, GaussCovariance = curve_fit(Gaussian,xValues,yValues,p0=[max(yValues),10,30],bounds = ([0,0,0],[1.2*max(yValues),1000,500]),maxfev=1000000)
print('mean='+str(round(GaussParameters[1],3))+'+-'+str(round(GaussParameters[2],3))+' , Max: '+str(round(GaussParameters[0],3)))

In [ ]:
LogNormParameters, LogNormCovariance = curve_fit(LogNorm,xValues,yValues,p0=[max(yValues),5,1],maxfev=1000000)
print('a='+str(round(LogNormParameters[0],1))+'| mu = '+str(round(LogNormParameters[1],2))+' | sigma = '+str(round(LogNormParameters[2],3)))
print('mode = '+str(round(np.exp(LogNormParameters[1]-LogNormParameters[2]**2),2)))

In [ ]:
def computeUncertaintyPositionsOfLogNormFit(xValues,LogNormParameters,RatioOfMax):
    Positions = np.zeros(2)
    Mode = np.exp(LogNormParameters[1]-LogNormParameters[2]**2)
    Max = LogNorm(Mode,*LogNormParameters)
    LeftPos = int(Mode)
    while (LogNorm(LeftPos,*LogNormParameters) > Max*RatioOfMax and LeftPos > xValues[0]):
        LeftPos -= 1
    Positions[0] = LeftPos
    RightPos = int(Mode)
    while (LogNorm(RightPos,*LogNormParameters) > Max*RatioOfMax and RightPos < xValues[-1]):
        RightPos += 1
    Positions[1] = RightPos
    return Positions

In [ ]:
LogNormUncertaintyPositions = computeUncertaintyPositionsOfLogNormFit(xValues,LogNormParameters, 0.1)
print(LogNormUncertaintyPositions)

In [ ]:
Fig, Ax = plt.subplots(1,1,squeeze=False,figsize=(5,5))
Ax[0,0].plot(xValues, yValues,'.')
Ax[0,0].plot(xValues,Gaussian(xValues,*GaussParameters))
Ax[0,0].plot(xValues,LogNorm(xValues,*LogNormParameters))
Ax[0,0].vlines(LogNormUncertaintyPositions[0],np.min(yValues),np.max(yValues),color = 'k', alpha = 0.5)
if (LogNormUncertaintyPositions[1] < np.max(xValues)):
    Ax[0,0].vlines(LogNormUncertaintyPositions[1],np.min(yValues),np.max(yValues),color = 'k', alpha = 0.5)

## Automatized data analysis

In [ ]:
def computeResults():
    Temperatures = []
    NASeries = []
    TotalNumberOfParticles = []
    NumberOfDataPoints = 0
    DirectoryPath = getDirectoryPathFromDialog()
    for Directory in glob.glob(DirectoryPath+'/*'):
        FileName = glob.glob(Directory+'/NA_Series*')[0]
        NASeries.append(ReadSeries(FileName))
        Temperatures.append(float(re.search(r'(?<=T=)[\d]*\.[\d]*',FileName).group()))
        TotalNumberOfParticles.append(float(re.search(r'(?<=N=)[\d]*',FileName).group()))
        NumberOfDataPoints += 1
    FractionsOfA = np.zeros(NumberOfDataPoints)
    Deviations = np.zeros((2,NumberOfDataPoints))
    for i in range(NumberOfDataPoints):
        NumberOfAAverage, AverageNumberDeviation, NumberIndex = computeEquilibriumAverage(NASeries[i])
        yValues, xValues = computeHist(NASeries[i])
        LogNormParameters, LogNormCovariance = curve_fit(LogNorm,xValues,yValues,p0=[max(yValues),5,1],maxfev=1000000)
        LogNormUncertaintyPositions = computeUncertaintyPositionsOfLogNormFit(xValues,LogNormParameters, 0.1)
        Mode = np.exp(LogNormParameters[1]-LogNormParameters[2]**2)
        print('T=',Temperatures[i],':')
        print('a='+str(round(LogNormParameters[0],1))+'| mu = '+str(round(LogNormParameters[1],2))+' | sigma = '+str(round(LogNormParameters[2],3)))
        print('mode = '+str(round(Mode,2))+" , uncertainty positions: "+str(LogNormUncertaintyPositions))
        FractionsOfA[i] = Mode/TotalNumberOfParticles[i]
        Deviations[0,i] = (Mode - LogNormUncertaintyPositions[0])/TotalNumberOfParticles[i]
        Deviations[1,i] = (LogNormUncertaintyPositions[1] - Mode)/TotalNumberOfParticles[i]
    return Temperatures, FractionsOfA, Deviations

In [ ]:
Temperatures, FractionsOfA, Deviations = computeResults()

In [ ]:
Fig, Host = plt.subplots(1,1,squeeze=False,figsize=(10,5))
Host[0,0].errorbar(FractionsOfA,Temperatures,yerr=None,xerr=Deviations,fmt='.',capsize=5)
Host[0,0].set_xlim(0,0.7)
Host[0,0].set_ylim(0.5,0.72)

In [ ]:
def readInResults():
    Temperatures = []
    NASeries = []
    TotalNumberOfParticles = []
    NumberOfDataPoints = 0
    DirectoryPath = getDirectoryPathFromDialog()
    for Directory in sorted(glob.glob(DirectoryPath+'/*')):
        FileName = glob.glob(Directory+'/NA_Series*')[0]
        NASeries.append(ReadSeries(FileName))
        Temperatures.append(float(re.search(r'(?<=T=)[\d]*\.[\d]*',FileName).group()))
        TotalNumberOfParticles.append(float(re.search(r'(?<=N=)[\d]*',FileName).group()))
        NumberOfDataPoints += 1
    C = np.zeros((len(Temperatures),int(TotalNumberOfParticles[0]+1)))
    xEdges = np.linspace(0,TotalNumberOfParticles[0]+1,num = int(TotalNumberOfParticles[0]+1))/TotalNumberOfParticles[0]
    yEdges = np.array(Temperatures)
    for i in range(NumberOfDataPoints):
        NumberOfAAverage, AverageNumberDeviation, NumberIndex = computeEquilibriumAverage(NASeries[i])
        yValues, xValues = computeHist(NASeries[i][NumberIndex:])
        C[i,xValues[0]:xValues[-1]+1] = yValues
        C[i,int(TotalNumberOfParticles[0])-(xValues[-1]+1):int(TotalNumberOfParticles[0])-xValues[0]] = yValues[::-1]
    return xEdges,yEdges,C

In [ ]:
xEdges,yEdges,C = readInResults()

In [ ]:
Fig, Ax = plt.subplots(1,1,squeeze=False,figsize=(10,5))
plot = Ax[0,0].pcolormesh(xEdges,yEdges,C,shading='nearest',cmap='hot')
Fig.colorbar(plot)

## Plot of used potential

In [ ]:
rCut = 2.5
NumberOfPoints = 500
rValues = np.linspace(0.95,3.0,num=NumberOfPoints)
def Potential(r):
    return 4.0*(r**(-12) - r**(-6) - rCut**(-12) + rCut**(-6) - (r-rCut)*(-12.0*rCut**(-13)+6.0*rCut**(-7)))
PotValues = np.zeros(NumberOfPoints)
for i in range(NumberOfPoints):
    PotValues[i] = Potential(rValues[i])

In [ ]:
Fig, Host = plt.subplots(1,1,squeeze=False)
Host[0,0].plot(rValues,PotValues)
host[0,0].set_xlabel('$x_A$')
host[0,0].set_ylabel(r'temperature $\left[\frac{\epsilon_{AA}}{k_{B}}\right]$')